In [3]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Activation, Input, Concatenate, Multiply
from tensorflow.keras.metrics import binary_accuracy
from tensorflow.keras.losses import binary_crossentropy
from sklearn.model_selection import train_test_split
from tensorflow.keras import backend as K
from sklearn.metrics import accuracy_score
from tensorflow.keras import callbacks
from tensorflow.keras.callbacks import ModelCheckpoint

tf.compat.v1.disable_eager_execution()

## DataSet/EXAMPLE_DATASET/final_comparison_dataset_neural.txt
## DataSet/EXAMPLE_DATASET/result_to_neural_250.txt 
## bu dizindeki data setler girilmelidir.
train_path = "/content/result_to_neural_250.txt" 
test_path = "/content/final_comparison_dataset_neural.txt" 


In [5]:
def read_input_file(file_path):
    weights = []
    values = []
    capacities = []
    best_picks = []

    with open(file_path, 'r') as file:
        lines = file.readlines()
        for i in range(0, len(lines), 4):
            # Okunan satırları boşluklarla ayrılan değerlere ayırarak diziye atama
            value_line  = list(map(int, lines[i].split()))
            weight_line = list(map(int, lines[i + 1].split()))
            capacity_line = list(map(int, lines[i + 2].split()))
            best_picks_line = list(map(int, lines[i + 3].split()))
            

            weights.append(weight_line)
            values.append(value_line)
            capacities.append(capacity_line)
            best_picks.append(best_picks_line)

    return weights, values, capacities, best_picks

def create_knapsack_dataset(file_path, count):
    weights, values, capacities, best_picks = read_input_file(file_path)
    x = [[],[]]
    y = [[]]
    max_price = 0
    for i in range(count):
        
        item_weights = weights[i]
        item_values = values[i]
        item_capacity = capacities[i]
        item_count = len(item_weights)

        # Max price değerlerini güncelleme
        max_price = max(max_price, np.max(item_values))
        
        # Normalize prices ve weights hesaplama
        normalized_prices = [price / max_price for price in item_values]
        normalized_weights = [weight / item_capacity[0] for weight in item_weights]


        x[0].append(normalized_weights)
        x[1].append(normalized_prices)
        y[0].append(best_picks[i])


    return x, y

In [6]:

X_train, y_train = create_knapsack_dataset(train_path,10000)
X_test, y_test = create_knapsack_dataset(test_path,2000)


In [7]:
def metric_overprice(input_prices):
    def overpricing(y_true, y_pred):
        y_pred = K.round(y_pred)
        return K.mean(K.batch_dot(y_pred, input_prices, 1) - K.batch_dot(y_true, input_prices, 1))

    return overpricing
    
def metric_space_violation(input_weights):
    def space_violation(y_true, y_pred):
        y_pred = K.round(y_pred)
        return K.mean(K.maximum(K.batch_dot(y_pred, input_weights, 1) - 1, 0))
    return space_violation


In [8]:
earlystopping = callbacks.EarlyStopping(monitor="loss",
                                        mode="min", patience=3,
                                        restore_best_weights=True)

def getResults(model):
    train_results = model.evaluate(X_train, y_train, 100, 0)
    test_results = model.evaluate(X_test, y_test, 100, 0);
    print("Model results(Train/Test):")
    print(f"Loss:               {train_results[0]:.2f} / {test_results[0]:.2f}")
    print(f"Binary accuracy:    {train_results[1]:.2f} / {test_results[1]:.2f}")
    print(f"Space violation:    {train_results[2]:.2f} / {test_results[2]:.2f}")
    print(f"Overpricing:        {train_results[3]:.2f} / {test_results[3]:.2f}")

## One layer supervised


In [18]:
def one_layer(item_count=250):
    input_shape = (item_count,)
    input_weights = Input(shape=input_shape)
    input_prices = Input(shape=input_shape)
    inputs_concat = Concatenate()([input_weights, input_prices])
    picks = Dense(item_count, use_bias=False, activation="sigmoid")(inputs_concat)
    model = Model(inputs=[input_weights, input_prices], outputs=[picks])
    model.compile("adam",
                  binary_crossentropy,
                  metrics=[binary_accuracy, metric_space_violation(input_weights),
                           metric_overprice(input_prices)])
    return model
model = one_layer(250)
dot_img_file = '/tmp/model_1.png'
tf.keras.utils.plot_model(model, to_file=dot_img_file, show_shapes=True)

Eğer eğitilmiş ve kaydedilmiş ağırlık verisi varsa fit aşamasını geçebilirsiniz.

In [ ]:
model.fit(X_train, y_train, epochs=256, verbose=0, callbacks=[ModelCheckpoint("one_layer.h5", monitor="loss", save_best_only=True, save_weights_only=True),earlystopping])

In [19]:
model.load_weights("/content/one_layer.h5")

In [20]:
getResults(model)

Model results(Train/Test):
Loss:               0.33 / 0.36
Binary accuracy:    0.85 / 0.83
Space violation:    0.26 / 0.28
Overpricing:        7.13 / 7.22


## Two layer supervised


In [21]:
def two_layer(item_count=250):
    input_shape = (item_count,)
    input_weights = Input(shape=input_shape)
    input_prices = Input(shape=input_shape)
    inputs_concat = Concatenate()([input_weights, input_prices])
    picks = Dense(item_count * 10, use_bias=False, activation="sigmoid")(inputs_concat)
    picks = Dense(item_count, use_bias=False, activation="sigmoid")(picks)
    model = Model(inputs=[input_weights, input_prices], outputs=[picks])
    model.compile("adam",
                  binary_crossentropy,
                  metrics=[binary_accuracy, metric_space_violation(input_weights),
                           metric_overprice(input_prices)])
    return model
model = two_layer(250)
dot_img_file = '/tmp/model_2.png'
tf.keras.utils.plot_model(model, to_file=dot_img_file, show_shapes=True)

In [ ]:
model.fit(X_train, y_train, epochs=96, verbose=0, callbacks=[ModelCheckpoint("two_layer.h5", monitor="loss", save_best_only=True, save_weights_only=True),earlystopping])

In [22]:
model.load_weights("/content/two_layer.h5")

In [23]:
getResults(model)

Model results(Train/Test):
Loss:               0.15 / 0.30
Binary accuracy:    0.95 / 0.86
Space violation:    0.07 / 0.18
Overpricing:        2.13 / 4.24
